In [ ]:
import sys
import os
import subprocess
import re
import numpy as np
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as psql
import getpass
import matplotlib as mpl
import argparse
import glob
import traceback
import hashlib
import math
import collections
import functools

from statsmodels.stats.proportion import proportion_confint as statmodels_proportion_confint

# import sklearn.preprocessing
# import sklearn.feature_selection
# import sklearn.ensemble 
# import sklearn.neural_network
# import sklearn.model_selection
# import sklearn.metrics
# import sklearn.pipeline
from sklearn.externals import joblib

mpl.rcParams['figure.dpi'] = 80

import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display
from slugify import slugify

In [ ]:
app_base_dir = '/home/spbproc/euso-spb-patt-reco-v1'
if app_base_dir not in sys.path:
    sys.path.append(app_base_dir)

import event_processing_v3
import event_processing_v4
import postgresql_v3_event_storage
import dataset_query_functions_v3

import tool.acqconv
from data_analysis_utils import *
from data_analysis_utils_dataframes import *
from data_analysis_utils_performance import *
# import supervised_classification as supc    

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [ ]:
# inverse_means_map = np.load('/home/spbproc/euso-spb-patt-reco-v1/resources/inverse_flat_average_directions_4m_flipud.npy')

In [ ]:
classification_id = '20190628_2'
model_data_snippets_dir = 'ver4_machine_learning_w_labeled_flight_' + classification_id
utah_file_analysis_snippets_dir = 'utah_events_directory_analysis'
data_snippets_dir = 'ver4_machine_learning_utah_classification_gtu_overlap_' + classification_id

subset_classification_slug = '_v4_ml_go_' + classification_id
# event_classification_v4_ml_go_20190628_2

os.makedirs(data_snippets_dir, exist_ok=True)

## Selecting the data

In [ ]:
event_processing_cls = event_processing_v4.EventProcessingV4
event_v3_storage_provider_utah = dataset_query_functions_v3.build_event_v3_storage_provider(
    event_storage_provider_config_file=os.path.join(app_base_dir,'config.ini'), 
    table_names_version='ver4',
    event_storage_class=postgresql_v3_event_storage.PostgreSqlEventV3StorageProvider,
    event_processing_class=event_processing_cls,
    readonly=False,
    schema_name_overwrite = 'miso2_2_spb_processing_v4'
)

query_functions_utah = dataset_query_functions_v3.Ver3DatasetQueryFunctions(event_v3_storage_provider_utah)

### Columns

In [ ]:
rfecv_selector_on_extra_trees__column_names = []

columns_list_file_pathname = os.path.join(model_data_snippets_dir, 'rfecv_selector_on_extra_trees__column_names.txt')
print(columns_list_file_pathname)
with open(columns_list_file_pathname, 'r') as columns_list_file:
    rfecv_selector_on_extra_trees__column_names = columns_list_file.read().splitlines()

In [ ]:
rfecv_selector_on_extra_trees__column_names__special = []

special_columns_list_file_pathname = os.path.join(model_data_snippets_dir, 'rfecv_selector_on_extra_trees__column_names__special.txt')
print(special_columns_list_file_pathname)
with open(special_columns_list_file_pathname, 'r') as special_columns_list_file:
    rfecv_selector_on_extra_trees__column_names__special = special_columns_list_file.read().splitlines()

In [ ]:
# This should be empty for now
rfecv_selector_on_extra_trees__column_names__special

In [ ]:
utah_columns_for_analysis_dict = query_functions_utah.get_columns_for_classification_dict__by_excluding(
    excluded_columns_re_list=('^.+$',),
    default_excluded_columns_re_list=[],
    included_columns_re_list=[('^$','source_file_(acquisition|trigger)(_full)?|global_gtu|packet_id|gtu_in_packet|event_id|num_gtu'),] + rfecv_selector_on_extra_trees__column_names
)

classification_utah_columns_for_analysis_dict = query_functions_utah.get_columns_for_classification_dict__by_excluding(
    excluded_columns_re_list=('^.+$',),
    default_excluded_columns_re_list=[],
    included_columns_re_list=rfecv_selector_on_extra_trees__column_names
)

print_columns_dict(utah_columns_for_analysis_dict)

WARNING: not selecting NULL trg lines

### Constructing the query

In [ ]:
current_columns_for_analysis_dict = utah_columns_for_analysis_dict

utah_select_clause_str, utah_tables_list = \
    query_functions_utah.get_query_clauses__select({
    **current_columns_for_analysis_dict,
})

utah_clauses_str = \
    query_functions_utah.get_query_clauses__join(utah_tables_list)

utah_source_data_type_num = 2

utah_where_clauses_str = ''
# ''' 
#     AND abs(gtu_in_packet-42) < 20
#     AND {database_schema_name}.event_orig_x_y.count_nonzero > 256*6
# '''

for table, cols_list in classification_utah_columns_for_analysis_dict.items():
    for col in cols_list:
        utah_where_clauses_str += ' AND {}.{} IS NOT NULL\n'.format(table, col)

utah_events_selection_query = query_functions_utah.get_events_selection_query_plain(
    source_data_type_num=utah_source_data_type_num,
    select_additional=utah_select_clause_str, 
    join_additional=utah_clauses_str,
    where_additional=utah_where_clauses_str,
    order_by='{data_table_name}.event_id', 
    offset=0, limit=1000000,
    base_select='')

In [ ]:
# print(utah_events_selection_query)

In [ ]:
utah_df = psql.read_sql(utah_events_selection_query, event_v3_storage_provider_utah.connection)

In [ ]:
len(utah_df)

In [ ]:
utah_df['dist_gtu_40'] = np.abs(utah_df['gtu_in_packet'] - 40)

In [ ]:
utah_df.head()

In [ ]:
utah_df[rfecv_selector_on_extra_trees__column_names].head()

In [ ]:
np.count_nonzero(utah_df['event_id'].isnull())

In [ ]:
utah_df['had_nan_fields'] = utah_df[rfecv_selector_on_extra_trees__column_names].isnull().any(axis=1)

In [ ]:
np.count_nonzero(utah_df['had_nan_fields'])

In [ ]:
utah_df_nonan = utah_df[~utah_df['had_nan_fields']]

In [ ]:
len(utah_df_nonan)

In [ ]:
# TODO
# SELECT COUNT(*) FROM spb_processing_v4_flatmap.event JOIN spb_processing_v4_flatmap.event_orig_x_y USING(event_id) WHERE source_data_type_num = 1 AND abs(gtu_in_packet-42) < 20 AND spb_processing_v4_flatmap.event_orig_x_y.count_nonzero > 256*6 LIMIT 5;
# SELECT COUNT( DISTINCT (source_file_acquisition, packet_id)) FROM spb_processing_v4_flatmap.event JOIN spb_processing_v4_flatmap.event_orig_x_y USING(event_id) WHERE source_data_type_num = 1 AND abs(gtu_in_packet-42) < 20 AND spb_processing_v4_flatmap.event_orig_x_y.count_nonzero > 256*6 LIMIT 5;


In [ ]:
utah_df_nonan.gtu_in_packet.hist(bins=128+1, figsize=(10,4))
plt.show()

# Applying models

This model does not use scaled data

In [ ]:
# standard_scaler_on_train_rfecv_columns_pathname = \
#      os.path.join(model_data_snippets_dir, 'standard_scaler_on_train_rfecv_columns.pkl')
# standard_scaler_on_train_rfecv_columns = joblib.load(standard_scaler_on_train_rfecv_columns_pathname)

In [ ]:
# flight__rfecv_columns_scaled_X = \
#     standard_scaler_on_train_rfecv_columns.transform(
#         flight_df[rfecv_selector_on_extra_trees__column_names].values)
# if np.count_nonzero(flight_df['had_nan_fields']) > 0:
#     flight_nonan__rfecv_columns_scaled_X = \
#         standard_scaler_on_train_rfecv_columns.transform(
#             flight_df_nonan[rfecv_selector_on_extra_trees__column_names].values)
# else:
#     flight_nonan__rfecv_columns_scaled_X = flight__rfecv_columns_scaled_X

## Extra trees classifier

In [ ]:
utah_rfecv_columns__X  = utah_df_nonan[rfecv_selector_on_extra_trees__column_names].values

In [ ]:
extra_trees_cls_on_train_rfecv__model_plk_pathname = \
    os.path.join(model_data_snippets_dir, 'extra_trees_cls_on_train_rfecv.pkl')
extra_trees_cls_on_train_rfecv_est = joblib.load(extra_trees_cls_on_train_rfecv__model_plk_pathname)

In [ ]:
# TODO RENAME
#       - extc_trn_rfecv_dn_est
#       - extc_trn_rfecv_dn_proba

cls_column_base = 'extra_trees_cls_on_train_rfecv_est'
cls_column = cls_column_base + '_dropna'
cls_proba_column = 'extra_trees_cls_on_train_rfecv_est_dropna_proba'

In [ ]:
# this might not be correct (but for this particular selection it should be fine becaus utah_df_nonan == utah_df )
utah_df[cls_column_base] = \
    extra_trees_cls_on_train_rfecv_est.predict(utah_rfecv_columns__X)

utah_df[cls_column] = \
    ((utah_df[cls_column_base]==1) & ~utah_df['had_nan_fields']).astype('int8')

In [ ]:
utah_df[cls_proba_column] = np.nan
utah_df.loc[utah_df['event_id'].isin(utah_df_nonan['event_id']), cls_proba_column] = \
    extra_trees_cls_on_train_rfecv_est.predict_proba(utah_rfecv_columns__X)[:,1]

In [ ]:
fig, ax = plt.subplots(figsize=(15,4))
# , bins=50+1, range=(20, 70)
h = ax.hist(utah_df_nonan.gtu_in_packet, bins=128+1, range=(0,128), label='All data')
ax.hist(utah_df_nonan[utah_df[cls_column] == 1].gtu_in_packet, bins=h[1], label='Classified "air shower"')
ax.legend()
fig.savefig(os.path.join(data_snippets_dir,'gtu_in_packet_hist,svg'), dpi=150)
plt.show()

### Updating `flight_df_nonan` with classification predictions
(not in the report)

In [ ]:
utah_df_nonan = utah_df[~utah_df['had_nan_fields']]

### Statistics of selected events

In [ ]:
np.count_nonzero(utah_df[cls_column])

In [ ]:
np.count_nonzero(utah_df[cls_column])/len(utah_df)

In [ ]:
utah_df[[cls_column, cls_proba_column]].describe()

In [ ]:
for proba in np.arange(0.5, 1.0, 0.1):
    print('p > {:.2f}: {}'.format(proba, np.count_nonzero(utah_df[cls_proba_column] > proba)))

In [ ]:
utah_df_nonan[rfecv_selector_on_extra_trees__column_names].head()

### Saving utah data into tsv
(not in the report)

In [ ]:
save_utah_data_dump_file = True
overwrite_utah_data_dump_file = False

In [ ]:
# if save_utah_data_dump_file:
#     utah_data_tsv_pathname = os.path.join(data_snippets_dir, 'utah_data.tsv.gz')

#     if overwrite_utah_data_dump_file or not os.path.exists(utah_data_tsv_pathname):
#         print('Saving', utah_data_tsv_pathname)
#         utah_df.to_csv(utah_data_tsv_pathname, sep='\t', compression='gzip')
#     else:
#         print('Already exists', utah_data_tsv_pathname)

# Visualization of the events

In [ ]:
shower_pred_by_proba_desc = utah_df_nonan[utah_df_nonan[cls_column] == 1].sort_values(cls_proba_column, ascending=False)
shower_pred_by_proba_asc = utah_df_nonan[utah_df_nonan[cls_column] == 1].sort_values(cls_proba_column, ascending=True)
noise_pred_by_proba_desc = utah_df_nonan[utah_df_nonan[cls_column] == 0].sort_values(cls_proba_column, ascending=False)
noise_pred_by_proba_asc = utah_df_nonan[utah_df_nonan[cls_column] == 0].sort_values(cls_proba_column, ascending=True)

### Visualization of tracks - air shower prediction - sorted by probability descending

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc, 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[1000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[5000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[10000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[20000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[30000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_desc[40000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

### Visualization of tracks - air shower prediction - sorted by probability ascending

In [ ]:
len(shower_pred_by_proba_asc)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[:20], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[4000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[8000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[10000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    shower_pred_by_proba_asc[10000:].query('dist_gtu_40 < 5'), 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

### Visualization of tracks - noise prediction - sorted by probability descending

In [ ]:
len(noise_pred_by_proba_desc)

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc, 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

**Shower events (20190628)**:
 - 676813 strong track  - should not been rejected

Shower events (20190409):
 - 1929 (short track, looks as single gtu track)
 
 661289

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc[100:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

Shower events (20190409):
 - 686894 (obvious air shower track)
 - 316757 (short obvious air shower track)

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc[200000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    noise_pred_by_proba_desc[100000:], 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

# Numbers of selected packets from eusospb analisi table

In [ ]:
eusospb_analisi_with_pathnames_left_pathname = os.path.join(utah_file_analysis_snippets_dir, 'eusospb_analisi_with_pathnames_left.tsv')
eusospb_analisi_with_pathnames_left_df = pd.read_csv(eusospb_analisi_with_pathnames_left_pathname, sep='\t', index_col=0)

In [ ]:
eusospb_analisi_with_pathnames_left_df

In [ ]:
df = eusospb_analisi_with_pathnames_left_df
pack_max_100 = np.min([df['pack'], np.ones(len(df))*100], axis=0)

for cfg in ['ta_euso', 'ta_euso_10', 'euso_bal', 'euso_bal_10', 'euso_bal_20']:
    df['max_' + cfg] = np.max([df[cfg + '_ec2'], df[cfg + '_ec5'], df[cfg + '_ec8']], axis=0)
    df['eff_' + cfg] = df['max_' + cfg] / pack_max_100

In [ ]:
eusospb_analisi_with_pathnames_left_df

## Counts of recognized showers

In [ ]:
# investigation of track gtu range - reuslt: 30 < gtu_in_packet < 68
#3793 5 32
# vis_events_df(
# #     utah_df_nonan.query('gtu_in_packet == 68').sort_values(cls_proba_column, ascending=False), 
#     utah_df_nonan.query('event_id == 364599'),
#     events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
#     close_after_vis=False, show=True, 
#     additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
#     by_one=True,
#     extension_func=None,
#     single_proj_width=4, single_proj_height=3
# )

In [ ]:
vis_events_df(
    utah_df.query('event_id == 735810'), 
    events_per_figure=10, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

### Counting

In [ ]:
gtu_40_pack_range = (34, 50) # 30, 68


df = eusospb_analisi_with_pathnames_left_df

for class_num, class_label in enumerate(['noise', 'shower']):
    df['num_' + class_label + '_events'] = -1
    df['num_' + class_label + '_events_gtu_40'] = -1
    df['num_' + class_label + '_packets'] = -1
    df['num_' + class_label + '_packets_gtu_40'] = -1
        
for k, file_pathname in df['file_pathname'].items():
    print(file_pathname)
    for class_num, class_label in enumerate(['noise', 'shower']):

        
#         df['num_' + class_label + '_multiple_event_packets'] = -1
#         df['num_' + class_label + '_multiple_packet_events'] = -1
#         df['num_' + class_label + '_multiple_packet_events_gtu_40'] = -1

        cls_events = utah_df_nonan[(utah_df_nonan['source_file_acquisition'] == file_pathname) & (utah_df_nonan[cls_column] == class_num)]
        
        cls_packets_num_events = cls_events.groupby('packet_id')['event_id'].count()
        num_packets = len(cls_packets_num_events)
        
        cls_events_gtu_40 = cls_events[
            ((cls_events.gtu_in_packet -4 + cls_events.num_gtu) > 40)  & 
            (cls_events.gtu_in_packet <= 45)
         ]
        
#         cls_events_gtu_40_pack = cls_events_gtu_40 \
#             .reset_index() \
#             .sort_values(['packet_id', 'dist_gtu_40'], ascending=True) \
#             .groupby('packet_id', as_index=False).first() \
#             .set_index('index')
        
        # number of events in gtu 40 packet
        cls_packets_num_events_gtu_40 = cls_events_gtu_40.groupby('packet_id')['event_id'].count()
        # number of packets
        num_packets_gtu_40 = len(cls_packets_num_events_gtu_40)
        # packets ids where thre are multiple suitable events around gtu 40
        multple_event_packets_gtu_40_indexes = cls_packets_num_events_gtu_40[cls_packets_num_events_gtu_40 > 1].index
        
        if(len(multple_event_packets_gtu_40_indexes) > 1):
            print('MULTIPLE EVENTS IN A SINGLE PACKET\n\t{}({}) / {}({}): {}'.format(
                class_label, class_num, file_pathname, k, len(multple_event_packets_gtu_40_indexes),
                # 'cls_packets_num_events_gtu_40[cls_packets_num_events_gtu_40 > 1]'
            ))
        
        print('\t{:<6} tot_pack={:<3d} gtu_40_pack={:<3d} pack={:<3d} max_euso_bal={:<3d} cls_events={:<4d} cls_events_gtu_40={:<4d}'.format(
            class_label,  num_packets, num_packets_gtu_40, df.loc[k].pack, df.loc[k].max_euso_bal, len(cls_events), len(cls_events_gtu_40)))
        
        
        if class_label == 'shower' and len(cls_packets_num_events) > len(cls_packets_num_events_gtu_40):
#             print(cls_packets_num_events.index)
#             print(cls_packets_num_events_gtu_40.index)
            gtu_40_missing_packets = cls_packets_num_events[~cls_packets_num_events.index.isin(cls_packets_num_events_gtu_40.index)]
            
            cls_events_outside_gtu_40_df = cls_events[cls_events.packet_id.isin(gtu_40_missing_packets.index)]
            
            print('\tOUTSIDE GTU=40\t{}({}): num_out={}'.format(
                class_label, class_num, len(cls_events_outside_gtu_40_df)
            ))
            print('\t\t{:<8} {:<3} {:<5} {:<3} {}'.format('event_id', 'pck', 'gtu.p', 'num', 'proba'))
            for ti, (tk, tr) in enumerate(cls_events_outside_gtu_40_df.sort_values('dist_gtu_40').iterrows()):
                print('\t\t{:<8} {:<3} {:<5} {:<3} {:.2f}'.format(tr['event_id'], tr['packet_id'], tr['gtu_in_packet'], tr['num_gtu'], tr[cls_proba_column]))
                if ti > 10:
                    break
            
        df.loc[k, 'num_' + class_label + '_events'] = len(cls_events)
        df.loc[k, 'num_' + class_label + '_events_gtu_40'] = len(cls_events_gtu_40)
        df.loc[k, 'num_' + class_label + '_packets'] = num_packets
        df.loc[k, 'num_' + class_label + '_packets_gtu_40'] = num_packets_gtu_40
        

In [ ]:
# MAIN MODIFICATION !!! <strike>df['pack'] == 100 always</strike>
# Changing back to pack - there are not 100 packets in a file

pack_max_100 = np.min([df['pack'], np.ones(len(df))*100], axis=0)
ones = np.ones(len(df))

df = eusospb_analisi_with_pathnames_left_df

for col in ['shower_packets', 'shower_packets_gtu_40']:
    df['eff_' + col] = np.min([df['num_' + col] / pack_max_100, ones], axis=0)
    
df['shower_contamination'] =  df['num_shower_events'] / df['num_shower_packets_gtu_40']
df['outside_gtu_40_packets'] = df['num_shower_packets'] - df['num_shower_packets_gtu_40']
df['total_contamination'] =  (df['num_shower_events'] + df['num_noise_events']) / df['pack']

In [ ]:
eusospb_analisi_with_pathnames_left_df[[
    'num_lens', 'run', 'per', 'mj', 'pack', 
    # 'euso_bal_ec2', 'euso_bal_ec5', 'euso_bal_ec8', 
    'max_euso_bal', 
    'eff_euso_bal', 
    'num_noise_events', 
    'num_noise_events_gtu_40', 
    'num_noise_packets', 
    'num_noise_packets_gtu_40',
    'num_shower_events', 
    'num_shower_events_gtu_40',
    'num_shower_packets', 
    'num_shower_packets_gtu_40',
    'eff_shower_packets_gtu_40', 
    'outside_gtu_40_packets',
    'shower_contamination', 'total_contamination',
    'pack', 'acq_group', 'file_pathname'
]].sort_values('mj', ascending=True)

In [ ]:
eusospb_analisi_with_pathnames_left_df.query('num_noise_packets == 0 and num_shower_packets == 0')

### Statistics by acq group

In [ ]:
def acq_group_stats(num_lens, acq_group=None, err_lims=(0.2,) ):
    
    
    print('Num lens:', num_lens)
    print('Acquisitions group:', acq_group)
    
    eusospb_analisi_2_len_by_mj = \
        eusospb_analisi_with_pathnames_left_df \
            .query('num_lens == {num_lens} and (num_noise_events > 0 or num_shower_events > 0) {acq_group_part}' \
                       .format(num_lens=num_lens, acq_group_part='and acq_group=="{}"'.format(acq_group) if acq_group else '')
                  ) \
            .sort_values('mj', ascending=True)
    
    print('Number of entries:', len(eusospb_analisi_2_len_by_mj))
    
    IPython.display.display(eusospb_analisi_2_len_by_mj[[
        'num_lens', 'run', 'per', 'mj', 'pack', 
        'max_euso_bal', 'max_euso_bal_10', 
        'num_noise_packets', 'num_shower_packets', 
        'num_noise_packets_gtu_40', 'num_shower_packets_gtu_40',
        'eff_euso_bal_10', 'eff_shower_packets_gtu_40',
        'acq_group', 'file_pathname'
    ]])
    
    
    pack_max_100 = np.min([eusospb_analisi_2_len_by_mj['pack'], np.ones(len(eusospb_analisi_2_len_by_mj))*100], axis=0)
    num_shower_packets_gtu_40 = eusospb_analisi_2_len_by_mj['num_shower_packets_gtu_40'].data

    mj_vals = eusospb_analisi_2_len_by_mj['mj']
    
    eff_euso_bal = eusospb_analisi_2_len_by_mj['eff_euso_bal']
#     eff_euso_bal_10 = eusospb_analisi_2_len_by_mj['eff_euso_bal_10'].data
    eff_shower_packets_gtu_40 = eusospb_analisi_2_len_by_mj['eff_shower_packets_gtu_40']
        
    yerr = statmodels_proportion_confint(
        np.min([num_shower_packets_gtu_40, 
                np.ones(len(eusospb_analisi_2_len_by_mj))*100], axis=0), 
#       np.ones(len(eusospb_analisi_2_len_by_mj))*100,
#       eusospb_analisi_2_len_by_mj['pack'].data,         # resored from 100
        pack_max_100,
        method='beta')
    
    # filtering
    
    for max_err in [None] + list(err_lims):
        t_mj_vals = mj_vals
        t_yerr = yerr
        t_eff_euso_bal = eff_euso_bal
#         t_eff_euso_bal_10 = eff_euso_bal_10
        t_eff_shower_packets_gtu_40 = eff_shower_packets_gtu_40
        max_err_suffix = ''
        t_pack_max_100 = pack_max_100
        
        if max_err is not None:
            max_err_suffix = '_max_err_{:.2f}'.format(max_err)
            yerr_mask = (yerr[1] - yerr[0])  < max_err
            t_mj_vals = np.array(mj_vals)[yerr_mask]
            t_yerr = [yerr[0][yerr_mask], yerr[1][yerr_mask]]
            t_eff_euso_bal = np.array(eff_euso_bal)[yerr_mask]
#             t_eff_euso_bal_10 = eff_euso_bal_10[yerr_mask]
            t_eff_shower_packets_gtu_40 = np.array(eff_shower_packets_gtu_40)[yerr_mask]
            t_pack_max_100 = pack_max_100[yerr_mask]
        
        fig, ax = plt.subplots(figsize=(4,3))
        ax.plot(t_mj_vals, t_eff_euso_bal, label='EUSO Bal. (P=1,R=1)')
#         ax.plot(mj_vals, eff_euso_bal_10, label='EUSO Bal. (P=1,R=1) -10%')
        ax.plot(t_mj_vals, t_eff_shower_packets_gtu_40, label='Classification model')  # .format(classification_id)) #  ({})

        ax.legend()
        ax.grid(axis='both', linestyle='--')
        x_range = t_mj_vals.max() - t_mj_vals.min()
        ax.set_xlim(t_mj_vals.min() - x_range*0.1, t_mj_vals.max() + x_range*0.1)
        ax.set_xlabel('Laser energy [mJ]')
        ax.set_ylabel('Efficiency')

        combined_ylim = ax.get_ylim()
        
        path_compatible_acq_group = slugify(acq_group) if acq_group else 'any'
        p = os.path.join(data_snippets_dir, 
                                 'efficiency_plot_acq_group_{}_eusobal_classification_model_comparison{}.svg' \
                                     .format(path_compatible_acq_group, str(max_err_suffix).replace('.','_')))
        print(p)
        fig.savefig(p)


        for mj, eff, m, ye1, ye2 in zip(t_mj_vals, t_eff_shower_packets_gtu_40, t_pack_max_100, *t_yerr):
            print('{:3.3f}\t{:3.3f}\t{:3.3f}\t{:3.3f}\t{:3.3f}'.format(mj, eff, m, ye1, ye2))

        fig, ax, errbr = \
            plot_efficiency_stat_simple(
                t_mj_vals, t_eff_shower_packets_gtu_40, 
                yerr=t_yerr,
                figsize=(4,3), ylabel='Efficiency', xlabel='Laser energy [mJ]',
                show=False
            )
        
        ax.set_ylim(combined_ylim)
        ax.set_xlim(t_mj_vals.min() - x_range*0.1, t_mj_vals.max() + x_range*0.1)
        ax.grid(axis='both', linestyle='--')
        
        p = os.path.join(data_snippets_dir, 
                                 'efficiency_plot_acq_group_{}_classification_model{}.svg' \
                                     .format(path_compatible_acq_group, str(max_err_suffix).replace('.','_')))
        print(p)
        fig.savefig(p)
        
        
        
        fig, ax = plt.subplots(figsize=(5,2.6))
        
        fig, ax, errbr = \
            plot_efficiency_stat_simple(
                t_mj_vals, t_eff_shower_packets_gtu_40, 
                yerr=t_yerr,
                figsize=None, ylabel='Efficiency', xlabel='Laser energy [mJ]',
                label='Extremly randomized trees classifier',
                show=False,
                ax=ax,
                errorbar_attrs={**EFFICIENCY_STAT_ERRORBAR_DEFAULTS, 'marker': '.'}
            )
        
        ax.plot(t_mj_vals, t_eff_euso_bal, label='First Level Trigger (P=1,R=1; 2 lens)', 
                color='red', marker='.', linestyle='-', alpha=.5,
                zorder=99)
        
#         ax.plot(mj_vals, eff_euso_bal_10, label='EUSO Bal. (P=1,R=1) -10%')
#         ax.plot(t_mj_vals, t_eff_shower_packets_gtu_40, label='Classification model')  # .format(classification_id)) #  ({})
        
        ax.legend()
        ax.grid(axis='both', linestyle='--')
        x_range = t_mj_vals.max() - t_mj_vals.min()
        ax.set_ylim(0.4,1.1)
        ax.set_xlim(t_mj_vals.min() - x_range*0.1, t_mj_vals.max() + x_range*0.1)
        ax.set_xlabel('Laser energy [mJ]')
        ax.set_ylabel('Efficiency')
        
        path_compatible_acq_group = slugify(acq_group) if acq_group else 'any'
        p = os.path.join(data_snippets_dir, 
                                 'efficiency_plot_acq_group_{}_eusobal_classifier_w_errbars_range_04_11_comparison{}_526.svg' \
                                     .format(path_compatible_acq_group, str(max_err_suffix).replace('.','_')))
        print(p)
        fig.savefig(p)
        
    
    plt.show()
    
    x_range = mj_vals.max() - mj_vals.min()

    print('Mean num packets outside GTU 40  ', eusospb_analisi_2_len_by_mj.outside_gtu_40_packets.mean())
    print('Median num packets outside GTU 40', eusospb_analisi_2_len_by_mj.outside_gtu_40_packets.median())
    
    fig, ax, errbr = \
        plot_efficiency_stat_simple(
            mj_vals, eusospb_analisi_2_len_by_mj.outside_gtu_40_packets,
            figsize=(4,3), ylabel='Num packets outside GTU 40', xlabel='Laser energy [mJ]', yerr=None,
            show=False
        )

    ax.set_xlim(mj_vals.min() - x_range*0.1, mj_vals.max() + x_range*0.1)
    ax.grid(axis='both', linestyle='--')

    p = os.path.join(data_snippets_dir, 
                             'outside_gtu_40_packets_plot_acq_group_{}_classification_model.svg' \
                                 .format(path_compatible_acq_group))
    print(p)
    fig.savefig(p)

    plt.show()
    
    print('Mean trig events / GTU40 packets  ', eusospb_analisi_2_len_by_mj.shower_contamination.mean())
    print('Median trig events / GTU40 packets', eusospb_analisi_2_len_by_mj.shower_contamination.median())
    print('Mean trig events /pack', eusospb_analisi_2_len_by_mj.total_contamination.mean())
    print('Median trig events /pack', eusospb_analisi_2_len_by_mj.total_contamination.median())
    
    fig, ax, errbr = \
        plot_efficiency_stat_simple(
            mj_vals, eusospb_analisi_2_len_by_mj.shower_contamination,
            figsize=(5,3), ylabel='Trig events / GTU40 packets', xlabel='Laser energy [mJ]', yerr=None,
            show=False
        )
    
    ax.set_xlim(mj_vals.min() - x_range*0.1, mj_vals.max() + x_range*0.1)
    ax.grid(axis='both', linestyle='--')

    p = os.path.join(data_snippets_dir, 
                             'contamination_plot_acq_group_{}_classification_model.svg' \
                                 .format(path_compatible_acq_group))
    print(p)
    fig.savefig(p)
    
    plt.show()
    
    
    cls_events = utah_df_nonan[utah_df_nonan['source_file_acquisition'].isin(eusospb_analisi_2_len_by_mj['file_pathname'])]

    cls_events_gtu_40 = cls_events[
        ((cls_events.gtu_in_packet -4 + cls_events.num_gtu) > 40)  & 
        (cls_events.gtu_in_packet <= 45)
     ]
    cls_events_gtu_40_pack = cls_events_gtu_40 \
        .reset_index() \
        .sort_values(['packet_id', 'dist_gtu_40'], ascending=True) \
        .groupby('packet_id', as_index=False).first() \
        .set_index('index')
    
    for proba in np.arange(0.5, 1.0, 0.1):
        print('p > {:.2f}: {}'.format(proba, np.count_nonzero(cls_events_gtu_40_pack[cls_proba_column] > proba)))
        
    fig, ax = plt.subplots(figsize=(4,3))
    cls_events_gtu_40_pack[cls_proba_column].hist(
        ax=ax, bins=100, alpha=1, range=(0,1))
    ax.set_ylabel('Number of events')
    ax.set_xlabel('Probability')
    ax.set_yscale('log')
    
    
    p = os.path.join(data_snippets_dir,  path_compatible_acq_group + '_events_gtu_40_pack_' + cls_proba_column +'_distribution_horizontal.svg')
    
    fig.savefig(p)
    
    plt.show()

In [ ]:
for num_lens in np.unique(eusospb_analisi_with_pathnames_left_df['num_lens']):
    for acq_group in (None, *eusospb_analisi_with_pathnames_left_df.query('num_lens=='+str(num_lens))['acq_group'].unique().tolist()):
        acq_group_stats(num_lens, acq_group)
        print()
        print('='*100)
        print()

### Low efficiency / failed entries

In [ ]:
def failed_mj(num_lens, acq_group=None, find_events=True, show_events=False, events_per_figure=10, gtu_range=(30, 45)):
    
    print('Num lens:', num_lens)
    print('Acquisitions group:', acq_group)
    
    eusospb_analisi_2_len_by_mj = \
        eusospb_analisi_with_pathnames_left_df \
            .query('num_lens == {num_lens} and (num_noise_events > 0 or num_shower_events > 0) {acq_group_part} and eff_shower_packets_gtu_40 < 0.5 ' \
                       .format(num_lens=num_lens, acq_group_part='and acq_group=="{}"'.format(acq_group) if acq_group else '')
                  ) \
            .sort_values('mj', ascending=True)
    
    print('Number of entries:', len(eusospb_analisi_2_len_by_mj))
    
    IPython.display.display(eusospb_analisi_2_len_by_mj[[
        'num_lens', 'run', 'per', 'mj', 'pack', 
        'max_euso_bal', 'max_euso_bal_10', 
        'num_noise_packets', 'num_shower_packets', 
        'num_noise_packets_gtu_40', 'num_shower_packets_gtu_40',
        'eff_euso_bal_10', 'eff_shower_packets_gtu_40',
        'acq_group', 'file_pathname'
    ]])
    
    if not find_events:
        return

    for i, r in eusospb_analisi_2_len_by_mj.iterrows():
        file_pathname = r['file_pathname']
        print(file_pathname)
        print('\teff_euso_bal = {:.3f}\n\teff_euso_bal_10 = {:.3f}\n\teff_shower_packets_gtu_40 = {:.3f}\n\tmj = {:.3f}\n\tpack = {:d}'.format(
            r['eff_euso_bal'], r['eff_euso_bal_10'], r['eff_shower_packets_gtu_40'], r['mj'], r['pack']
        ))
#         all_file_entries = eusospb_analisi_with_pathnames_left_df.query('source_file_acquisition == "{}"'.format(file_pathname))
#         print('Number of all entries:', len(all_file_entries))
        
        acq_events = utah_df_nonan[(utah_df_nonan['source_file_acquisition'] == file_pathname)]
        
        for class_num, class_label in enumerate(['noise', 'shower']):
            
            cls_events = acq_events[(acq_events[cls_column] == class_num)]
            cls_packets_num_events = cls_events.groupby('packet_id')['event_id'].count()
            
            cls_events_gtu_40_df = \
                cls_events[
                    (gtu_range[0] < cls_events['gtu_in_packet']) & \
                    (cls_events['gtu_in_packet'] < gtu_range[1])
                ].sort_values(cls_proba_column, ascending=bool(class_num))
            
            cls_packets_num_events_gtu_40 = cls_events_gtu_40_df.groupby('packet_id')['event_id'].count()
            
            print()
            print(
                'Events classified as {class_label}:\n\tcount:        {count:5d} ({gtu_range[0]:d} < gtu_in_packet < {gtu_range[1]:d})\n' \
                '\tpacket count: {packet_count:5d} ({gtu_range[0]:d} < gtu_in_packet < {gtu_range[1]:d})' \
                    .format(
                    class_label=class_label, 
                    count=len(cls_events_gtu_40_df), 
                    packet_count=len(cls_packets_num_events_gtu_40),
                    gtu_range=gtu_range
                )
            )
            print()
            
            if show_events:
                vis_events_df(
                    cls_events_gtu_40_df, 
                    events_per_figure=events_per_figure, max_figures=1, vis_gtux=True, vis_gtuy=True, 
                    close_after_vis=False, show=True, 
                    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
                    by_one=True,
                    extension_func=None,
                    single_proj_width=4, single_proj_height=3
                )
            print('-'*100)
    

In [ ]:
for num_lens in np.unique(eusospb_analisi_with_pathnames_left_df['num_lens']):
    for acq_group in eusospb_analisi_with_pathnames_left_df.query('num_lens=='+str(num_lens))['acq_group'].unique().tolist():
        failed_mj(num_lens, acq_group, find_events=True, show_events=True)
        print()
        print('='*100)
        print()
        

## Examination of `300916/GLS/allpackets-SPBEUSO-ACQUISITION-20161004-041003-001.001--45degaway36per.root`

- conclusion: all seems to be fine
- file has 39 packets (as pack column)
- 35 packet are correctly recognized showers (as note in the file)
- 2 packets are noise and are misclassified as an air shower

In [ ]:
eusospb_analisi_with_pathnames_left_df[eusospb_analisi_with_pathnames_left_df.file_pathname == '300916/GLS/allpackets-SPBEUSO-ACQUISITION-20161004-041003-001.001--45degaway36per.root']

In [ ]:
cls_events_df = utah_df_nonan[(utah_df_nonan['source_file_acquisition'] == '300916/GLS/allpackets-SPBEUSO-ACQUISITION-20161004-041003-001.001--45degaway36per.root')]
# & (utah_df_nonan[cls_column] == class_num)

In [ ]:
cls_events_df

In [ ]:
len(cls_events_df.groupby('packet_id'))

In [ ]:
positive_cls_events_df = cls_events_df[(cls_events_df[cls_column] == 1)]

In [ ]:
len(positive_cls_events_df)

In [ ]:
positive_cls_events_gtu_40_df = positive_cls_events_df[(positive_cls_events_df.gtu_in_packet < 45) & (positive_cls_events_df.gtu_in_packet >= 30)]

In [ ]:
positive_cls_events_gtu_non_40_df = positive_cls_events_df[(positive_cls_events_df.gtu_in_packet >= 45) | (positive_cls_events_df.gtu_in_packet < 30)]

In [ ]:
len(positive_cls_events_gtu_40_df)

In [ ]:
len(positive_cls_events_gtu_non_40_df)

In [ ]:
vis_events_df(
    positive_cls_events_gtu_40_df, 
    events_per_figure=40, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

In [ ]:
vis_events_df(
    positive_cls_events_gtu_non_40_df, 
    events_per_figure=40, max_figures=1, vis_gtux=True, vis_gtuy=True, 
    close_after_vis=False, show=True, 
    additional_printed_columns=[cls_proba_column, 'source_file_acquisition_full'],
    by_one=True,
    extension_func=None,
    single_proj_width=4, single_proj_height=3
)

## Examination of `041016/GLS/allpackets-SPBEUSO-ACQUISITION-20161004-041003-001.001--45degaway36per.root`

In [ ]:
f = '041016/GLS/allpackets-SPBEUSO-ACQUISITION-20161004-041003-001.001--45degaway36per.root'

In [ ]:
eusospb_analisi_with_pathnames_left_df[eusospb_analisi_with_pathnames_left_df.file_pathname == f]

In [ ]:
cls_events_df = utah_df_nonan[(utah_df_nonan['source_file_acquisition'] == f)]

In [ ]:
cls_events_df

# Acquisition groups for all files

In [ ]:
files_acq_groups_pathname = os.path.join(utah_file_analysis_snippets_dir, 'files_acq_groups.tsv')
files_acq_groups_df = pd.read_csv(files_acq_groups_pathname, sep='\t', index_col=0)

In [ ]:
files_acq_groups_df.columns.tolist()

In [ ]:
utah_df_nonan_w_acq_groups_w_mj_df = \
    pd.merge(
        pd.merge(utah_df_nonan, files_acq_groups_df, how='outer', left_on=['source_file_acquisition'], right_on=['file_pathname']),
        eusospb_analisi_with_pathnames_left_df[['file_pathname', 'num_lens', 'mj', 'pack']],
        how='left',
        on=['file_pathname'], 
    )

In [ ]:
utah_df_nonan_w_acq_groups_w_mj_df[[
    'source_file_acquisition', 
     'entries',
    cls_column,
    'acq_group_l0', 'acq_group_l1', 'acq_group_l2', 'run', 
    'num_lens', 'mj'
]][(~utah_df_nonan_w_acq_groups_w_mj_df['source_file_acquisition'].isnull()) & 
   (~utah_df_nonan_w_acq_groups_w_mj_df[cls_column].isnull())] # 'pack', 'entries',

In [ ]:
utah_df_nonan_w_acq_groups_w_mj_df[~utah_df_nonan_w_acq_groups_w_mj_df.mj.isnull()]

In [ ]:
utah_classified_entries_mask = (~utah_df_nonan_w_acq_groups_w_mj_df['source_file_acquisition'].isnull()) & (~utah_df_nonan_w_acq_groups_w_mj_df[cls_column].isnull())
utah_df_nonan_w_acq_groups_w_mj_classied_df = utah_df_nonan_w_acq_groups_w_mj_df[utah_classified_entries_mask]

In [ ]:
print('acq_group_l0', np.count_nonzero(utah_df_nonan_w_acq_groups_w_mj_classied_df.acq_group_l0.isnull()))
print('acq_group_l1', np.count_nonzero(utah_df_nonan_w_acq_groups_w_mj_classied_df.acq_group_l1.isnull()))
print('acq_group_l2', np.count_nonzero(utah_df_nonan_w_acq_groups_w_mj_classied_df.acq_group_l2.isnull()))
print('run', np.count_nonzero(utah_df_nonan_w_acq_groups_w_mj_classied_df.run.isnull()))
print('num_lens', np.count_nonzero(utah_df_nonan_w_acq_groups_w_mj_classied_df.num_lens.isnull()))
print('mj', np.count_nonzero(utah_df_nonan_w_acq_groups_w_mj_classied_df.mj.isnull()))
print('entries', np.count_nonzero(utah_df_nonan_w_acq_groups_w_mj_classied_df.entries.isnull()))

# Saving the classification results

### Preparation of data for the database

In [ ]:
utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df = \
    utah_df_nonan_w_acq_groups_w_mj_classied_df[[
        'event_id',
        'source_file_acquisition', 
         'entries',
        cls_column,
        cls_proba_column,
        'acq_group_l0', 'acq_group_l1', 'acq_group_l2', 'run', 
        'num_lens', 'mj'
    ]].copy()

In [ ]:
# len\
(
np.unique( ((utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df.entries // 128) // 50) * 50 )
)

In [ ]:
utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df['packets_approx'] = \
    ((utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df.entries // 128) // 50) * 50

In [ ]:
numeric_fillna_columns = ['num_lens', 'mj']
str_fillna_columns = ['acq_group_l1', 'acq_group_l2']

utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df[numeric_fillna_columns] = \
    utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df[numeric_fillna_columns].fillna(-1)

utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df[str_fillna_columns] = \
    utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df[str_fillna_columns].fillna('')

In [ ]:
utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df

In [ ]:
subset_classification_slug

### Saving to the database

In [ ]:
classification_columns = collections.OrderedDict([
    (event_v3_storage_provider_utah.data_table_pk, int),
    (cls_column, int), 
    (cls_proba_column, float),
    ('source_file_acquisition', str), 
    ('entries', int),
    ('packets_approx', int),
    ('acq_group_l0', str), 
    ('acq_group_l1', str), 
    ('acq_group_l2', str), 
    ('run', int), 
    ('num_lens', int), 
    ('mj', float)
])

print(
    event_v3_storage_provider_utah.connection.cursor().mogrify(
        event_v3_storage_provider_utah.get_classification_table_query(subset_classification_slug, classification_columns.items())
    ).decode()
)

event_v3_storage_provider_utah.create_classification_table(subset_classification_slug, classification_columns.items())

event_v3_storage_provider_utah.save_classification_data(
    subset_classification_slug, 
    utah_df_nonan_w_acq_groups_w_mj_classied_for_db_df[list(classification_columns.keys())].values, 
    classification_columns.items(), 
    num_inserts_at_once=1000, morgify=False
)

In [ ]:
# classification_column_names = [col for col in flight_df_gtu_36_45_pack_nonan.columns \
#                                if col.startswith('tsne_') or \
#                                        col == cls_column or col == cls_proba_column or \
#                                        col == event_v3_storage_provider_flight.data_table_pk]
# classification_column_type = [(int \
#                                if col == cls_column or col.endswith('dbscan_y') or \
#                                        col == event_v3_storage_provider_flight.data_table_pk or \
#                                        col == event_v3_storage_provider_flight.data_table_pk \
#                                    else float) \
#                               for col in classification_column_names]

# classification_columns = list(zip(classification_column_names, classification_column_type))


# print('-'*50)
# print(subset_classification_slug)
# print(classification_column_names)

# event_v3_storage_provider_flight.create_classification_table(subset_classification_slug, classification_columns)

# event_v3_storage_provider_flight.save_classification_data(
#     subset_classification_slug, 
#     flight_df_gtu_36_45_pack_nonan[classification_column_names].values, classification_columns, 
#     num_inserts_at_once=1000, morgify=False
# )

In [ ]:
#event_v3_storage_provider_flight.connection.reset()

In [ ]:
# flight_nonan_classified_shower_pathname = os.path.join(data_snippets_dir, 'flight_nonan_classified shower.tsv')

In [ ]:
# flight_df_nonan[flight_df_nonan['extra_trees_cls_on_train_kbest400_128_est_dropna']==1].to_csv(flight_nonan_classified_shower_pathname, sep='\t')

In [ ]:
# TODO select clusters with positive classification (sort by the number of classifications) show distribution of event types

In [ ]:

# flight_df_nonan_subset[['dbscan_tsne_y','manual_classification_class_number'].hist('dbscan_tsne_y', figsize=(24,4), bins=2*len(dbscan_on_tsne_classes)+1)

# plt.show()

In [ ]:
# THIS IS NOT WHAT IS DESIRED - values should be split into features ?
# flight_nonan__cls_tsneclu_corr_df = \
#     flight_df_nonan[['dbscan_tsne_y', 'manual_classification_class_number']].corr()
# f, ax = plt.subplots(figsize=(28,22))
# plt.close('all')
# sns.heatmap(flight_nonan__cls_tsneclu_corr_df, cmap='inferno', annot=True)
# plt.show()

In [ ]:
#     f, ax = plt.subplots()
#     f.set_size_inches(8,4)
#     flight_df_nonan_subset[['dbscan_tsne_y', 'manual_classification_class_number']].plot.bar(by='dbscan_tsne_y', ax=ax)
    

In [ ]:
# flight_nonan__tsne__gmm_y_pred = gmm.predict(flight_df_nonan[['tsne_X_0','tsne_X_1']].values)

In [ ]:
# flight_data__k50best_var_th_scaled_X = \
#     k50best_f_classif_selector_on_var_th_sc_train.transform(
#         var_th_selector_on_scaled_train.transform(
#             standard_scaler_on_train.transform(
#                 unl_flight_df[analyzed_common_df_columns].dropna().values)
#         )
#     )

# extra_trees_classifier_on_train_kbest50__X_flight = flight_data__k50best_var_th_scaled_X
# extra_trees_classifier_on_train_kbest50__y_flight_pred = \
#     extra_trees_classifier_on_train_kbest50.predict(extra_trees_classifier_on_train_kbest50__X_flight)